<a href="https://colab.research.google.com/github/BhojRajBist/BhojRajBist/blob/main/Async_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install asyncio

In [1]:
async def my_function():
    await some_other_function()


In [4]:
import asyncio

async def greet():
    print("Hi, waiting for 2 seconds...")
    await asyncio.sleep(2)
    print("Done waiting!")

# Run the async function
await greet()


Hi, waiting for 2 seconds...
Done waiting!


In [5]:
async def task(name, delay):
    print(f"{name} started...")
    await asyncio.sleep(delay)
    print(f"{name} finished after {delay} seconds")

# Run them together
await asyncio.gather(
    task("Task 1", 2),
    task("Task 2", 3),
    task("Task 3", 1),
)


Task 1 started...
Task 2 started...
Task 3 started...
Task 3 finished after 1 seconds
Task 1 finished after 2 seconds
Task 2 finished after 3 seconds


[None, None, None]

In [6]:
import httpx

async def fetch_joke():
    async with httpx.AsyncClient() as client:
        res = await client.get("https://official-joke-api.appspot.com/jokes/random")
        print(res.json())

await fetch_joke()


/usr/local/lib/python3.11/dist-packages/httpx/_client.py:13: RuntimeWarning: coroutine 'say_hello' was never awaited
  from ._auth import Auth, BasicAuth, FunctionAuth


{'type': 'general', 'setup': 'Why did the mushroom get invited to the party?', 'punchline': 'Because he was a fungi.', 'id': 35}


In [7]:
async def fetch_multiple_jokes(n=3):
    async with httpx.AsyncClient() as client:
        tasks = [client.get("https://official-joke-api.appspot.com/jokes/random") for _ in range(n)]
        responses = await asyncio.gather(*tasks)
        for i, res in enumerate(responses):
            print(f"Joke {i+1}: {res.json()['setup']} - {res.json()['punchline']}")

await fetch_multiple_jokes(5)


Joke 1: What did the mountain climber name his son? - Cliff.
Joke 2: Why are oranges the smartest fruit? - Because they are made to concentrate. 
Joke 3: I started a new business making yachts in my attic this year... - The sails are going through the roof.
Joke 4: What do you call a fly without wings? - A walk.
Joke 5: Knock knock. 
 Who's there? 
 Hatch. 
 Hatch who? - Bless you!
